# Sub-SBML Breakout Session #2

## _Ayush Pandey_


Problem:

Implement an advanced diffusion model (two-step or three-step) and use it as a membrane model for a System of your choice.

In [2]:
# Sketch solution

# Create a two-step diffusion model with the following reactions:
# A_external + M <-> complex_M_A, kb1, kd1
# complex_M_A --> complex_M_A_activated, irreversible, k_mature
# complex_M_A_activated <-> A_internal + M, kb2, kd2


from subsbml import createNewSubsystem, SimpleModel, SimpleReaction
import libsbml
IPTG_mb_ss = createNewSubsystem()
model = IPTG_mb_ss.createNewModel('A_membrane_two_step','second','mole','substance')

IPTG_mb = SimpleModel(model)

per_second = IPTG_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)

conc_per_time = IPTG_mb.createNewUnitDefinition('conc_per_time',[libsbml.UNIT_KIND_SECOND, 
                                                                 libsbml.UNIT_KIND_LITRE, libsbml.UNIT_KIND_MOLE],
                                                [-1, 1, -1],0,1)
substance = IPTG_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

IPTG_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
IPTG_mb.createNewCompartment('external','external',1e-6,'litre',True)


IPTG_mb.createNewSpecies( 'A','external',0,False,'substance')
IPTG_mb.createNewSpecies( 'A','internal',0,False,'substance')
IPTG_mb.createNewSpecies( 'M','internal',1,False,'substance')
IPTG_mb.createNewSpecies( 'complex_M_A','internal',0,False,'substance')
IPTG_mb.createNewSpecies( 'complex_M_A_activated','internal',0,False,'substance')

IPTG_mb.createNewParameter( 'kb1',0.01,False,'conc_per_time')
IPTG_mb.createNewParameter( 'kd1',0.2,False,'per_second')

IPTG_mb.createNewParameter( 'kb2',0.02,False,'conc_per_time')
IPTG_mb.createNewParameter( 'kd2',0.4,False,'per_second')

IPTG_mb.createNewParameter( 'k_mature',0.1,False,'per_second')

model = IPTG_mb_ss.getSBMLDocument().getModel()

r1 = SimpleReaction(model.createReaction())
r1.setId('r1')
r1.setReversible(True)
r1.createNewReactant('A',False,1)
r1.createNewReactant('M', False, 1)
r1.createNewProduct('complex_M_A', False, 1)
r1.createRate('kb * A * M - kd * complex_M_A')

# Finish writing all reactions



# Write to SBML file 
_ = IPTG_mb_ss.writeSBML('membrane_A_two_step.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


Now, use this membrane model in the code from the overview notebook

In [ ]:
# Create a more detailed IPTG membrane where IPTG binds to an intermediate transporter protein, forms a complex
# then transports out of the cell system to the external environment 

mb = createSubsystem('membrane_A_two_step.xml', membrane = True)

# Define ss1, ss2, mb
cell_3 =  System('cell_two_step',ListOfInternalSubsystems = [ss1],
                 ListOfExternalSubsystems = [ss2],
                 ListOfMembraneSubsystems = [mb])

cell_3_model = cell_3.getModel()
cell_3_model.setSpeciesAmount('IPTG', 1e4, compartment = 'cell_3_external')
cell_3_model.writeSBML('cell_3_model.xml')

combined_model = combineSystems([cell_1, cell_2, cell_3])

In [ ]:
# Run simulation code